In [1]:

import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import Dataset
import torch.optim as optim
from torchvision import transforms
import numpy as np
import torch
from torch import nn
import os
from torch.utils.data import DataLoader, random_split
from torch.nn import DataParallel
from torch.utils.data import Subset
import shutil
from codes.helpers import FocalLoss
import cv2

In [2]:
generator_name_to_load = "final_model.pth"

batch_size = 1
device = "cuda"
logdir = "logs/"

In [3]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os
import numpy as np


class Fashion_Data(Dataset):
    def __init__(self, folder_train, augmentation=None):
        self.folder_train = folder_train
        self.transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()]) if augmentation is None \
                            else transforms.Compose([augmentation, transforms.Resize((256, 256)), transforms.ToTensor()])
        
        self.filenames = [f for f in os.listdir(folder_train) if os.path.isfile(os.path.join(folder_train, f))]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name_train = os.path.join(self.folder_train, self.filenames[idx])

        img_train = Image.open(img_name_train).convert('RGB')
        img_train = self.transform(img_train)

        return img_train


In [4]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

test_folder = "logs/input/"
test_set = Fashion_Data(test_folder)
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=True)



In [5]:
# The diagram is provided to understand the architecture
from architecture.segnet import SegNet


generator = SegNet(in_channels=3).to(device)
generator.load_state_dict(torch.load(f'logs/{generator_name_to_load}'))

<All keys matched successfully>

In [6]:
# The code to make sure the model input and output are correct, we can test the first batch to fed into the network
data = next(iter(test_dataloader))

test_data = data
output_test = generator(test_data.to(device))
print(output_test.shape)

torch.Size([1, 1, 128, 128])


In [7]:
from tqdm import tqdm
from torchvision.utils import save_image

generator.eval()
pbar = tqdm(test_dataloader)
for idx, x in enumerate(pbar):
    x = x.to(device, dtype=torch.float32)
    output = generator(x)
    for j, gen_image in enumerate(output):
        save_image(gen_image, os.path.join("logs", "output", f"generated_{idx}_{j}.png"))

100%|██████████| 1/1 [00:00<00:00, 16.47it/s]


In [8]:
def categorize_and_recolor_image(image_path):
    img = Image.open(image_path)
    data = np.array(img)

    # Check if image has an alpha channel; if so, ignore it in processing
    if data.shape[2] == 4:
        data = data[:, :, :3]

    height, width, _ = data.shape
    # Initialize a mask to keep track of already processed (recategorized) pixels
    processed_mask = np.zeros((height, width), dtype=bool)

    threshold_rg = 5  # Threshold for R-G and R-B for red-ish, including browns and purples
    threshold_w = 220  # Threshold for identifying white-ish pixels
    threshold_yg = 5  # Additional threshold for yellow, ensuring G is close to R and B is low

    # Define functions for each color categorization to improve readability
    def apply_mask(condition, new_color):
        nonlocal processed_mask
        mask = condition & ~processed_mask
        data[mask] = new_color
        processed_mask |= mask

    # White-ish
    white_condition = (data[:, :, 0] > threshold_w) & (data[:, :, 1] > threshold_w) & (data[:, :, 2] > threshold_w)
    apply_mask(white_condition, [5, 5, 5])

    # Yellow-ish
    yellow_condition = (data[:, :, 0] > data[:, :, 2] + threshold_yg) & (np.abs(data[:, :, 0] - data[:, :, 1]) < threshold_yg)
    apply_mask(yellow_condition, [25, 25, 25])

    # Red-ish (including browns and purples)
    red_condition = (data[:, :, 0] > data[:, :, 1] + threshold_rg) & (data[:, :, 0] > data[:, :, 2] + threshold_rg)
    apply_mask(red_condition, [22, 22, 22])

    # Green-ish
    green_condition = (data[:, :, 1] > data[:, :, 0] + threshold_rg) & (data[:, :, 1] > data[:, :, 2] + threshold_rg)
    apply_mask(green_condition, [21, 21, 21])

    # Blue-ish
    blue_condition = (data[:, :, 2] > data[:, :, 0] + threshold_rg) & (data[:, :, 2] > data[:, :, 1] + threshold_rg)
    apply_mask(blue_condition, [24, 24, 24])
    
    
    # Convert back to PIL Image for output
    new_img = Image.fromarray(data)
    return new_img